<a href="https://colab.research.google.com/github/ryukkt62/ktkim/blob/main/(LSTM%2CDNN)%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%20%EC%A3%BC%EA%B0%80%20%EC%98%88%EC%B8%A1/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 구글 드라이브에 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# STEP 1: 데이터 로드 및 전처리

In [ ]:
# 데이터 다운 및 확인
path = '/content/drive/MyDrive/Colab Notebooks/딥러닝으로 걷는 시계열 예측/'
df1 = pd.read_csv(path+'data/kospi200.csv',encoding='cp949',index_col=0,
                  header=0, sep=',')
df2 = pd.read_csv(path+'data/samsung.csv',encoding='cp949',index_col=0,
                  header=0, sep=',')
display(df1)
display(df2)

,시가,고가,저가,현재가,거래량
일자,,,,,
2020-01-31,290.24,291.47,284.53,284.53,"101,455"
2020-01-30,293.27,294.11,287.09,288.37,"101,535"
2020-01-29,294.38,295.67,292.45,293.98,"85,731"
2020-01-28,294.98,296.30,291.30,292.77,"130,172"
2020-01-23,303.77,304.72,301.71,302.33,"86,908"
...,...,...,...,...,...
2018-05-11,318.03,319.70,317.86,318.51,"112,400"
2018-05-10,316.51,317.34,315.11,317.31,"120,900"
2018-05-09,315.93,316.08,312.60,314.42,"123,610"


,시가,고가,저가,종가,거래량
일자,,,,,
2020-01-31,"57,800","58,400","56,400","56,400","19,749,457"
2020-01-30,"58,800","58,800","56,800","57,200","20,821,939"
2020-01-29,"59,100","59,700","58,800","59,100","16,446,102"
2020-01-28,"59,400","59,400","58,300","58,800","23,664,541"
2020-01-23,"61,800","61,800","60,700","60,800","14,916,555"
...,...,...,...,...,...
2018-05-11,"52,000","52,200","51,200","51,300","10,314,997"
2018-05-10,"51,700","51,700","50,600","51,600","13,905,263"
2018-05-09,"52,600","52,800","50,900","50,900","16,128,305"


In [ ]:
# 데이터 파악,
# df1의 경우 거래량이 object로 돼있음
# df2의 경우 모든 데이터가 object로 돼있음
df1.info()
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 426 entries, 2020-01-31 to 2018-05-04
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시가      426 non-null    float64
 1   고가      426 non-null    float64
 2   저가      426 non-null    float64
 3   현재가     426 non-null    float64
 4   거래량     426 non-null    object 
dtypes: float64(4), object(1)
memory usage: 20.0+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 426 entries, 2020-01-31 to 2018-05-04
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시가      426 non-null    object
 1   고가      426 non-null    object
 2   저가      426 non-null    object
 3   종가      426 non-null    object
 4   거래량     426 non-null    object
dtypes: object(5)
memory usage: 20.0+ KB


In [ ]:
# 문자열 데이터를 숫자로 바꿔주기
def to_numeric(df):
    df = str(df).replace(',','')
    return df

# 콤마(,) 제거해주기
df1 = df1.applymap(to_numeric)
df2 = df2.applymap(to_numeric)
# object를 float형식으로 바꿔주기
df1 = df1.astype(float)
df2 = df2.astype(float)
# 데이터 최근순으로 바꿔주기
df1 = df1.sort_index()
df2 = df2.sort_index()

In [ ]:
df1.info()
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 426 entries, 2018-05-04 to 2020-01-31
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시가      426 non-null    float64
 1   고가      426 non-null    float64
 2   저가      426 non-null    float64
 3   현재가     426 non-null    float64
 4   거래량     426 non-null    float64
dtypes: float64(5)
memory usage: 20.0+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 426 entries, 2018-05-04 to 2020-01-31
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시가      426 non-null    float64
 1   고가      426 non-null    float64
 2   저가      426 non-null    float64
 3   종가      426 non-null    float64
 4   거래량     426 non-null    float64
dtypes: float64(5)
memory usage: 20.0+ KB


In [ ]:
df2.head(20)

,시가,고가,저가,종가,거래량
일자,,,,,
2018-05-04,53000.0,53900.0,51800.0,51900.0,39565391.0
2018-05-08,52600.0,53200.0,51900.0,52600.0,23104720.0
2018-05-09,52600.0,52800.0,50900.0,50900.0,16128305.0
2018-05-10,51700.0,51700.0,50600.0,51600.0,13905263.0
2018-05-11,52000.0,52200.0,51200.0,51300.0,10314997.0
2018-05-14,51000.0,51100.0,49900.0,50100.0,14909272.0
2018-05-15,50200.0,50400.0,49100.0,49200.0,18709146.0
2018-05-16,49200.0,50200.0,49150.0,49850.0,15918683.0
2018-05-17,50300.0,50500.0,49400.0,49400.0,10365440.0


In [ ]:
# 데이터 확인
display(df1)
display(df2)

,시가,고가,저가,현재가,거래량
일자,,,,,
2018-05-04,320.56,321.00,316.75,316.75,173911.0
2018-05-08,317.79,319.53,314.95,315.87,141252.0
2018-05-09,315.93,316.08,312.60,314.42,123610.0
2018-05-10,316.51,317.34,315.11,317.31,120900.0
2018-05-11,318.03,319.70,317.86,318.51,112400.0
...,...,...,...,...,...
2020-01-23,303.77,304.72,301.71,302.33,86908.0
2020-01-28,294.98,296.30,291.30,292.77,130172.0
2020-01-29,294.38,295.67,292.45,293.98,85731.0


,시가,고가,저가,종가,거래량
일자,,,,,
2018-05-04,53000.0,53900.0,51800.0,51900.0,39565391.0
2018-05-08,52600.0,53200.0,51900.0,52600.0,23104720.0
2018-05-09,52600.0,52800.0,50900.0,50900.0,16128305.0
2018-05-10,51700.0,51700.0,50600.0,51600.0,13905263.0
2018-05-11,52000.0,52200.0,51200.0,51300.0,10314997.0
...,...,...,...,...,...
2020-01-23,61800.0,61800.0,60700.0,60800.0,14916555.0
2020-01-28,59400.0,59400.0,58300.0,58800.0,23664541.0
2020-01-29,59100.0,59700.0,58800.0,59100.0,16446102.0


In [ ]:
# DataFrame형식을 array로 바꿔주기
df1 = df1.values
df2 = df2.values

In [ ]:
print(df1.shape)
print(df2.shape)
display(df1)
display(df2)

(426, 5)
(426, 5)


array([[   320.56,    321.  ,    316.75,    316.75, 173911.  ],
       [   317.79,    319.53,    314.95,    315.87, 141252.  ],
       [   315.93,    316.08,    312.6 ,    314.42, 123610.  ],
       ...,
       [   294.38,    295.67,    292.45,    293.98,  85731.  ],
       [   293.27,    294.11,    287.09,    288.37, 101535.  ],
       [   290.24,    291.47,    284.53,    284.53, 101455.  ]])

array([[   53000.,    53900.,    51800.,    51900., 39565391.],
       [   52600.,    53200.,    51900.,    52600., 23104720.],
       [   52600.,    52800.,    50900.,    50900., 16128305.],
       ...,
       [   59100.,    59700.,    58800.,    59100., 16446102.],
       [   58800.,    58800.,    56800.,    57200., 20821939.],
       [   57800.,    58400.,    56400.,    56400., 19749457.]])

In [ ]:
# array형식 데이터 저장
path2 ='/content/drive/MyDrive/Colab Notebooks/딥러닝으로 걷는 시계열 예측/'
np.save(path2+'data/kospi200.npy',arr=df1)
np.save(path2+'data/samsung.npy',arr=df2)